<a href="https://colab.research.google.com/github/threegenie/knn_project/blob/main/new_work_times_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os, sys
import numpy as np
from google.colab import drive

drive.mount('/content/drive')
# df1 = pd.read_csv('/content/drive/My Drive/df_2016.csv',encoding= 'unicode_escape')
# df2 = pd.read_csv('/content/drive/My Drive/df_2017.csv',encoding= 'unicode_escape')
# df3 = pd.read_csv('/content/drive/My Drive/df_2018.csv',encoding= 'unicode_escape')
# df4 = pd.read_csv('/content/drive/My Drive/df_2019.csv',encoding= 'unicode_escape')
df = pd.read_csv('/content/drive/My Drive/df_2020.csv',encoding= 'unicode_escape')


Mounted at /content/drive


In [3]:
# df = pd.concat([df4,df5],ignore_index=True)
# df.shape

(149726, 3)

In [3]:
df.head()

,Unnamed: 0,year,sentence
0,0,2020,The fugitive from Japan has put Lebanon in an ...
1,1,2020,Mixed reaction to the news that Maj. Gen. Qass...
2,2,2020,"In bronze, silver or aluminum, a statue is an ..."
3,3,2020,"Also this week, holiday fireworks in Iceland a..."
4,4,2020,"The teenager, accused in the death of the Barn..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69119 entries, 0 to 69118
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  69119 non-null  int64 
 1   year        69119 non-null  int64 
 2   sentence    69119 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.6+ MB


In [7]:
df = df.drop(['Unnamed: 0','year'],axis=1)

In [8]:
df.head()

,sentence
0,The fugitive from Japan has put Lebanon in an ...
1,Mixed reaction to the news that Maj. Gen. Qass...
2,"In bronze, silver or aluminum, a statue is an ..."
3,"Also this week, holiday fireworks in Iceland a..."
4,"The teenager, accused in the death of the Barn..."


#### 토큰 정제

In [9]:
!pip install squarify

In [10]:
import re
import squarify
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from spacy.tokenizer import Tokenizer

In [25]:
# #알파벳, 숫자만 받을 수 있도록 함수 작성(text):

# regex = r"[^a-zA-Z0-9 ]"
# subst = ""

# def tokenize(text):

#     # 정규식 적용
#     tokens = re.sub(regex, subst, text)

#     # split
#     tokens = tokens.split()
    
#     return tokens

# df['Tokens'] = df['sentence'].apply(tokenize)

In [11]:
#기본 불용어 불러오기
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
print(nlp.Defaults.stop_words)

{'sixty', 'too', 'must', 'they', 'herein', 'sometime', 'whole', 'else', 'am', 'across', 'latterly', '‘re', 'above', 'call', 'can', '‘s', 'even', 'keep', 'give', 'along', 'therefore', 'could', 'up', 'be', 'our', 'cannot', 'ourselves', '’ll', 'those', 'himself', 'herself', 'put', 'back', 'around', 'within', "'re", 'more', 'onto', 'her', 'seems', 'almost', 'five', 'whether', 'twenty', 'out', 'regarding', 'becoming', 'behind', 'his', 'twelve', 'quite', 'who', 'few', 'why', 'already', 'there', 'have', 'yet', 'many', "'m", 'nevertheless', 'itself', "n't", 'full', 'doing', '‘d', 'about', 'your', 'none', 'these', 'say', 'when', 'formerly', 'using', 'again', 'an', 'than', 'same', 'whereafter', '’d', 'go', 'ours', 'just', 'less', 'me', 'the', 'several', 'together', 'other', 'made', 'now', 'amongst', 'top', 'part', 'through', 'see', 'either', 'themselves', 'anyone', 'we', 'fifty', 'he', 'would', 'four', 'which', 'mostly', 'serious', 'only', 'how', 'always', 'also', 'whereby', '‘ve', 'n‘t', '’m', 

In [13]:
#불용어 추가하기
st_words = nlp.Defaults.stop_words.union(['this', 'i', 'it', 'is', 'in', 'and', 'a', 'the', 'but', 'to', 'the',
                                          'â\x80\x94','said','york','u.s.'])

In [28]:
# #불용어 처리 + lemmatization
# tokens = []
# regex = r"[^a-zA-Z0-9 ]"
# subst = ""

# for doc in tokenizer.pipe(df['sentence']):
#     doc_tokens = []
#     for token in doc:
#         if token.text.lower() not in st_words and ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
#             doc_tokens.append(token.text.lower())
#     tokens.append(doc_tokens)
 
# df['Tokens'] = tokens
# df['Tokens'].head(3)

0    [love,, political, change,, pennant:, readers,...
1    [director, barbet, schroeder, follows, ashin, ...
2    [eric, drouetâs, arrest,, second, role, prot...
Name: Tokens, dtype: object

In [14]:
def tokenize(text):
  text = text.lower()
  text = text.replace("\n"," ")

  subst = ""
  regex = "[^a-zA-Z0-9 ]"
  doc = re.sub(regex, subst, text)

  doc = nlp(doc)
  tokens = []

  for token in doc :
    if (token.text.lower() not in st_words) & (token.is_stop == False) & (token.is_punct == False) & (token.is_alpha):
      tokens.append(token.lemma_)
      
  return tokens

In [ ]:
df['Tokens']=df['sentence'].apply(tokenize)
df['Tokens'].head(3)

#### 단어 빈도에 따라 시각화

In [ ]:
def word_count(docs):
   
    # 전체 코퍼스에서 단어 빈도 카운트
    word_counts = Counter()

    # 단어가 존재하는 문서의 빈도 카운트, 단어가 한 번 이상 존재하면 +1
    word_in_docs = Counter()

    # 전체 문서의 갯수
    total_docs = len(docs)

    for doc in docs:
        word_counts.update(doc)
        word_in_docs.update(set(doc))

    temp = zip(word_counts.keys(), word_counts.values())

    wc = pd.DataFrame(temp, columns = ['word', 'count'])

    # 단어의 순위
    # method='first': 같은 값의 경우 먼저나온 요소를 우선
    wc['rank'] = wc['count'].rank(method='first', ascending=False)
    total = wc['count'].sum()

    # 코퍼스 내 단어의 비율
    wc['percent'] = wc['count'].apply(lambda x: x / total)

    wc = wc.sort_values(by='rank')

    # 누적 비율
    # cumsum() : cumulative sum
    wc['cul_percent'] = wc['percent'].cumsum()

    temp2 = zip(word_in_docs.keys(), word_in_docs.values())
    ac = pd.DataFrame(temp2, columns=['word', 'word_in_docs'])
    wc = ac.merge(wc, on='word')
    
    # 전체 문서 중 존재하는 비율
    wc['word_in_docs_percent'] = wc['word_in_docs'].apply(lambda x: x / total_docs)

    return wc.sort_values(by='rank')

In [ ]:
from collections import Counter
word_counts = Counter()
df['Tokens'].apply(lambda x: word_counts.update(x))
word_counts.most_common(10)

In [ ]:
wc = word_count(df['Tokens'])
wc.head(5)

In [ ]:
import squarify
import matplotlib.pyplot as plt

color=['viridis']

wc_top5 = wc[wc['rank'] <= 10]
squarify.plot(sizes=wc_top5['percent'], label=wc_top5['word'], alpha=0.6)
plt.axis('off')
plt.title('Token Top 10')
plt.show()

In [ ]:
sns.barplot(y=wc['word'][:10],x=wc['count'][:10],ax=ax[1],orient='h')

#### 코로나바이러스 관련 기사 찾기

In [ ]:
label = {}

for title in df.title.unique():
  title_low = title.lower()
  if 'senior' in title_low:
    label.setdefault(title,0)
  else:
    label.setdefault(title,1)